In [39]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
np.random.seed(666)

In [2]:
os.chdir('/Users/gloriatang/Downloads/')

In [3]:
data=pd.read_csv('modeling_data_v2.csv')
oot=pd.read_csv('oot_data_v2.csv')

In [4]:
data.columns
#data1=data1.drop(data1.columns[0:3],axis=1)

Index(['HASHED_ATOM_CUSTOMER_ID', 'OS_TYPE_e', 'PRODUCTION_ID', 'RATINGS',
       'MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER',
       'MOVIE_DETAILS_PAGES_VIEWED_BY_ALL_CUSTOMERS',
       'USER_SHARE_OF_ALL_MOVIE_DETAIL_PAGES', 'WANT_TO_SEE_CLICKS',
       'TRAILER_VIEWS', 'TICKETS',
       ...
       'count_HASHED_ATOM_CUSTOMER_ID_1d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_3d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_7d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_14d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_28d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_30d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_35d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_42d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_49d_ratings',
       'count_HASHED_ATOM_CUSTOMER_ID_56d_ratings'],
      dtype='object', length=119)

In [5]:
data.shape

(1781036, 119)

In [6]:
# Deal with categorical variables
## Drop useless columns

columns = ['TICKETS_CLASS','PRODUCTION_ID','RELEASE_DATE','DATE','HASHED_ATOM_CUSTOMER_ID','TICKETS']
y=data['TICKETS_CLASS']
X = data.drop(columns, axis=1)

In [7]:
print(X.shape, y.shape)

(1781036, 113) (1781036,)


In [8]:
## One-hot encoding for 'WEEKDAY_OF_ENGAGEMENT','OS_TYPE_e'
X_dummy=pd.get_dummies(X, prefix_sep='_',columns=['WEEKDAY_OF_ENGAGEMENT','OS_TYPE_e'])
## Convert DATE_TO_RELEASE to numeric
X_dummy["DATE_TO_RELEASE"]=X_dummy["DATE_TO_RELEASE"].str.replace('+','')
X_dummy["DATE_TO_RELEASE"]=X_dummy["DATE_TO_RELEASE"].str.replace(' days 00:00:00.000000000','').astype(int)

In [116]:
np.random.seed(666)
X_train, X_test, y_train,y_test = train_test_split(X_dummy, y, test_size = 0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1246725, 121) (1246725,)
(534311, 121) (534311,)


## Multi-Classification Model

### Naive Bayes

In [117]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()

In [118]:
#negative X not allowed
X_train_NB = X_train.drop(['DATE_TO_RELEASE'], axis=1)
X_test_NB = X_test.drop(['DATE_TO_RELEASE'], axis=1)
NB.fit(X_train_NB, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [119]:
# Predicting the test set results
y_pred_NB = NB.predict(X_test_NB)

y_pred_NB_name = np.vectorize(reversefactor.get)(y_pred_NB)
# Making the Confusion Matrix & show result
print(pd.crosstab(y_test_name, y_pred_NB_name, rownames=['Actual Value'], colnames=['Predicted Value']))
print('Accurancy:',NB.score(X_test_NB, y_test)*100 )

print(classification_report(y_test, y_pred_NB))

Predicted Value  Group Tickets  None  One Ticket  Two Tickets
Actual Value                                                 
Group Tickets              411   224         843        11085
None                     14536  7468       28996       389128
One Ticket                1665   876        3299        45706
Two Tickets               1031   524        1928        26591
Accurancy: 6.760669348
             precision    recall  f1-score   support

        0.0       0.73      0.02      0.03    440791
        1.0       0.13      0.09      0.11     51273
        2.0       0.05      0.78      0.09     29620
        3.0       0.11      0.15      0.13     12627

avg / total       0.62      0.07      0.04    534311



In [123]:
Weighted_F1 = (0.11*51273+0.09*29620+0.13*12627)/(51273+30074+12627)
print('Weighted F1:',Weighted_F1 )

Weighted F1: 0.10585204418243344


In [139]:
#trian
y_train_NB = NB.predict(X_train_NB)
print(classification_report(y_train, y_train_NB))

             precision    recall  f1-score   support

        0.0       0.72      0.01      0.03   1026842
        1.0       0.13      0.09      0.11    120166
        2.0       0.05      0.77      0.09     70102
        3.0       0.11      0.15      0.12     29615

avg / total       0.61      0.07      0.04   1246725



In [140]:
Weighted_train_NB = (0.11*120166+0.09*70102+0.12*29615)/(120166+70102+29615)
print('Weighted F1:',Weighted_train_NB )

Weighted F1: 0.1049705525211135


### Logistic Regression

In [120]:
# Feature Scaling， good for logistic regression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_l = scaler.fit_transform(X_train)
X_test_l = scaler.transform(X_test)

In [121]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [122]:
logreg.fit(X_train_l, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [124]:
# Predicting the Test set results
y_pred_log = logreg.predict(X_test_l)
#Reverse factorize (converting y_pred from 0,1,2 and 3)
definitions = ['None','One Ticket','Two Tickets','Group Tickets']
reversefactor = dict(zip(range(4),definitions))
y_test_name = np.vectorize(reversefactor.get)(y_test)
y_pred_log_name = np.vectorize(reversefactor.get)(y_pred_log)
# Making the Confusion Matrix
print(pd.crosstab(y_test_name, y_pred_log_name, rownames=['Actual Value'], colnames=['Predicted Value']))
print('Accurancy:',logreg.score(X_test_l, y_test)*100 )

# Evaluate model accuracy
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_log))

Predicted Value  Group Tickets    None  One Ticket  Two Tickets
Actual Value                                                   
Group Tickets               29    9929        2504          165
None                        37  436969        3513          272
One Ticket                  52   39100       11807          314
Two Tickets                 19   23194        6007          400
Accurancy: 84.0718233388
             precision    recall  f1-score   support

        0.0       0.86      0.99      0.92    440791
        1.0       0.50      0.23      0.31     51273
        2.0       0.35      0.01      0.03     29620
        3.0       0.21      0.00      0.00     12627

avg / total       0.78      0.84      0.79    534311



In [125]:
Weighted_F1_lg = (0.31*51273+0.03*29620+0.0*12627)/(51273+29620+12627)
print('Weighted F1:',Weighted_F1_lg )

Weighted F1: 0.1794613986313088


In [137]:
##train
y_train_log = logreg.predict(X_train_l)
print(classification_report(y_train, y_train_log))

             precision    recall  f1-score   support

        0.0       0.86      0.99      0.92   1026842
        1.0       0.49      0.23      0.31    120166
        2.0       0.39      0.01      0.03     70102
        3.0       0.24      0.00      0.00     29615

avg / total       0.78      0.84      0.79   1246725



In [138]:
Weighted_train_lg = (0.31*120166+0.03*70102+0.0*29615)/(120166+70102+29615)
print('Weighted F1:',Weighted_train_lg )

Weighted F1: 0.1789793662993501


### Random Forest

In [126]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 11, criterion = 'entropy', random_state = 0,verbose = 3)
rf.fit(X_train, y_train)

building tree 1 of 11


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.7s remaining:    0.0s


building tree 2 of 11


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.0s remaining:    0.0s


building tree 3 of 11
building tree 4 of 11
building tree 5 of 11
building tree 6 of 11
building tree 7 of 11
building tree 8 of 11
building tree 9 of 11
building tree 10 of 11
building tree 11 of 11


[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:  1.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=1,
            oob_score=False, random_state=0, verbose=3, warm_start=False)

In [145]:
# Predicting the test set results
y_pred_rf = rf.predict(X_test)

y_pred_rf_name = np.vectorize(reversefactor.get)(y_pred_rf)
# Making the Confusion Matrix & show result
print(pd.crosstab(y_test_name, y_pred_rf_name, rownames=['Actual Value'], colnames=['Predicted Value']))
print('Accurancy:',rf.score(X_test, y_test)*100 )

print(classification_report(y_test, y_pred_rf))

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    3.6s finished


Predicted Value  Group Tickets    None  One Ticket  Two Tickets
Actual Value                                                   
Group Tickets              960    4981        4672         2014
None                      2017  420876       11705         6193
One Ticket                2086   18607       23786         6794
Two Tickets               1193   13593       10160         4674


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    2.7s finished


Accurancy: 84.2760115364
             precision    recall  f1-score   support

        0.0       0.92      0.95      0.94    440791
        1.0       0.47      0.46      0.47     51273
        2.0       0.24      0.16      0.19     29620
        3.0       0.15      0.08      0.10     12627

avg / total       0.82      0.84      0.83    534311



In [128]:
Weighted_F1_rf = (0.47*51273+0.19*29620+0.1*12627)/(51273+29620+12627)
print('Weighted F1:',Weighted_F1_rf )

Weighted F1: 0.331360243798118


In [146]:
#train
y_train_rf = rf.predict(X_train)
print(classification_report(y_train, y_train_rf))

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    5.9s finished


             precision    recall  f1-score   support

        0.0       0.98      0.99      0.99   1026842
        1.0       0.93      0.89      0.91    120166
        2.0       0.95      0.85      0.89     70102
        3.0       0.96      0.85      0.90     29615

avg / total       0.97      0.97      0.97   1246725



In [142]:
Weighted_train_rf = (0.91*120166+0.89*70102+0.90*29615)/(120166+70102+29615)
print('Weighted F1:',Weighted_train_rf )

Weighted F1: 0.9022768472323918


### XGBoost

In [129]:
import xgboost as xgb
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

In [134]:
#setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['silent'] = 1
param['num_class'] = 4

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
y_pred_xgb = bst.predict(xg_test)


[0]	train-merror:0.140947	test-merror:0.139707
[1]	train-merror:0.137724	test-merror:0.136572
[2]	train-merror:0.137233	test-merror:0.136175
[3]	train-merror:0.136323	test-merror:0.135333
[4]	train-merror:0.135756	test-merror:0.134865


In [135]:
y_pred_xgb_name = np.vectorize(reversefactor.get)(y_pred_xgb)
# Making the Confusion Matrix & show result
print(pd.crosstab(y_test_name, y_pred_xgb_name, rownames=['Actual Value'], colnames=['Predicted Value']))

print(classification_report(y_test, y_pred_xgb))
acc = np.sum(y_pred_xgb == y_test) / y_test.shape[0]
print('Accurancy:',acc)

Predicted Value  Group Tickets    None  One Ticket  Two Tickets
Actual Value                                                   
Group Tickets               26    6984        5097          520
None                         1  437933        2609          248
One Ticket                   1   27694       22708          870
Two Tickets                  6   18179        9851         1584
             precision    recall  f1-score   support

        0.0       0.89      0.99      0.94    440791
        1.0       0.56      0.44      0.50     51273
        2.0       0.49      0.05      0.10     29620
        3.0       0.76      0.00      0.00     12627

avg / total       0.84      0.87      0.83    534311

Accurancy: 0.865134724907


In [136]:
Weighted_F1_xg = (0.5*51273+0.1*29620+0.0*12627)/(51273+29620+12627)
print('Weighted F1:',Weighted_F1_xg)

Weighted F1: 0.3058008982035928


In [143]:
y_train_xgb = bst.predict(xg_train)
print(classification_report(y_train, y_train_xgb))

             precision    recall  f1-score   support

        0.0       0.89      0.99      0.94   1026842
        1.0       0.56      0.44      0.49    120166
        2.0       0.52      0.06      0.10     70102
        3.0       0.78      0.00      0.00     29615

avg / total       0.84      0.86      0.83   1246725



In [144]:
Weighted_train_xgb = (0.49*120166+0.10*70102+0.0*29615)/(120166+70102+29615)
print('Weighted F1:',Weighted_train_xgb )

Weighted F1: 0.299666368022994


## Model performance in OOT data

### Data preparation

In [84]:
#Follow the same process

# Deal with categorical variables
## Drop useless columns
columns = ['TICKETS_CLASS','PRODUCTION_ID','RELEASE_DATE','DATE','HASHED_ATOM_CUSTOMER_ID','TICKETS']
oot_y=oot['TICKETS_CLASS']
oot_X = oot.drop(columns, axis=1)

## One-hot encoding for 'WEEKDAY_OF_ENGAGEMENT','OS_TYPE_e'
oot_X=pd.get_dummies(oot_X, prefix_sep='_',columns=['WEEKDAY_OF_ENGAGEMENT','OS_TYPE_e'])
## Convert DATE_TO_RELEASE to numeric
oot_X["DATE_TO_RELEASE"]=oot_X["DATE_TO_RELEASE"].str.replace('+','')
oot_X["DATE_TO_RELEASE"]=oot_X["DATE_TO_RELEASE"].str.replace(' days 00:00:00.000000000','').astype(int)

In [104]:
#NB
X_oot_NB = oot_X.drop(['DATE_TO_RELEASE'], axis=1)

y_oot_NB = NB.predict(X_oot_NB)
print(classification_report(oot_y, y_oot_NB))


             precision    recall  f1-score   support

        0.0       0.70      0.82      0.76    327793
        1.0       0.13      0.01      0.01     79457
        2.0       0.02      0.03      0.03     29107
        3.0       0.00      0.01      0.01      8682

avg / total       0.54      0.61      0.56    445039



In [130]:
Weighted_F1_oot_NB = (0.01*79457+0.03*29107+0.01*8682)/(79457+29107+8682)
print('Weighted F1:',Weighted_F1_oot_NB )

Weighted F1: 0.014965116080719171


In [77]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
oot_X = scaler.fit_transform(oot_X)


In [81]:
#logistic regression
y_oot_log = logreg.predict(oot_X)
print(classification_report(oot_y, y_oot_log))
print('Accurancy:',logreg.score(oot_X, oot_y)*100 )

             precision    recall  f1-score   support

        0.0       0.75      0.98      0.85    327793
        1.0       0.12      0.01      0.02     79457
        2.0       0.22      0.07      0.10     29107
        3.0       0.05      0.02      0.03      8682

avg / total       0.59      0.73      0.64    445039

Accurancy: 72.9488876256


In [131]:
Weighted_F1_oot_log = (0.02*79457+0.10*29107+0.03*8682)/(79457+29107+8682)
print('Weighted F1:',Weighted_F1_oot_log )

Weighted F1: 0.040600958668099554


In [89]:
#random forest

y_oot_rf = rf.predict(oot_X)
print(classification_report(oot_y, y_oot_rf))
print('Accurancy:',rf.score(oot_X, oot_y)*100 )

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.6s finished


             precision    recall  f1-score   support

        0.0       0.79      0.91      0.85    327793
        1.0       0.43      0.22      0.29     79457
        2.0       0.10      0.08      0.09     29107
        3.0       0.03      0.00      0.00      8682

avg / total       0.66      0.72      0.68    445039



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


Accurancy: 71.8164026074


[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.6s finished


In [132]:
Weighted_F1_oot_rf = (0.29*79457+0.09*29107+0.00*8682)/(79457+29107+8682)
print('Weighted F1:',Weighted_F1_oot_rf )

Weighted F1: 0.21887450318134521


In [83]:
#xgboost
xg_oot = xgb.DMatrix(oot_X, label=oot_y)
y_oot_xgb = bst.predict(xg_oot)
print(classification_report(oot_y, y_oot_xgb))
acc = np.sum(y_oot_xgb == oot_y) / oot_y.shape[0]
print('Accurancy:',acc)

             precision    recall  f1-score   support

        0.0       0.75      0.94      0.84    327793
        1.0       0.46      0.15      0.23     79457
        2.0       0.01      0.00      0.01     29107
        3.0       0.00      0.00      0.00      8682

avg / total       0.64      0.72      0.66    445039

Accurancy: 0.716633823103


In [133]:
Weighted_F1_oot_xg = (0.23*79457+0.01*29107+0.00*8682)/(79457+29107+8682)
print('Weighted F1:',Weighted_F1_oot_xg )

Weighted F1: 0.15835235317196322


# Next Step: Tune best model parameters

### RF Grid Search for parameters
1. **N_estimators**: n_estimators represents the number of trees in the forest. Usually the higher the number of trees the better to learn the data. However, adding a lot of trees can slow down the training process considerably
2. **max_depth**: max_depth represents the depth of each tree in the forest. The deeper the tree, the more splits it has and it captures more information about the data. 
3. **min_samples_split**: min_samples_split represents the minimum number of samples required to split an internal node. This can vary between considering at least one sample at each node to considering all of the samples at each node. When we increase this parameter, each tree in the forest becomes more constrained as it has to consider more samples at each node.
4. **min_samples_leaf**: min_samples_leaf is The minimum number of samples required to be at a leaf node. This parameter is similar to min_samples_splits, however, this describe the minimum number of samples of samples at the leafs, the base of the tree.
5. **max_features**: max_features represents the number of features to consider when looking for the best split.

In [148]:
print(X_train.shape,y_train.shape)

(1246725, 121) (1246725,)


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 210, num = 6)]

# Number of features to consider at every split
max_features = [round(x, 2) for x in np.linspace(start = 0.2, stop = 1, num = 5)]
max_features.append('sqrt')
## Empirical good default values are max_features=n_features for regression problems, 
## and max_features = sqrt(n_features) for classification tasks

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 50, num = 5)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 50, 90, 130, 170, 210], 'max_features': [0.20000000000000001, 0.40000000000000002, 0.59999999999999998, 0.80000000000000004, 1.0, 'sqrt'], 'max_depth': [5, 16, 27, 38, 50, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
RF = RandomForestClassifier()

# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = RF, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose = 2,
                               scoring = 'f1_weighted',
                               n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=90, min_samples_split=2, min_samples_leaf=4, max_features=0.2, max_depth=27, bootstrap=False 
[CV] n_estimators=90, min_samples_split=2, min_samples_leaf=4, max_features=0.2, max_depth=27, bootstrap=False 
[CV] n_estimators=90, min_samples_split=2, min_samples_leaf=4, max_features=0.2, max_depth=27, bootstrap=False 
[CV] n_estimators=90, min_samples_split=5, min_samples_leaf=2, max_features=0.2, max_depth=38, bootstrap=True 
[CV]  n_estimators=90, min_samples_split=5, min_samples_leaf=2, max_features=0.2, max_depth=38, bootstrap=True, total=25.9min
[CV] n_estimators=90, min_samples_split=5, min_samples_leaf=2, max_features=0.2, max_depth=38, bootstrap=True 
[CV]  n_estimators=90, min_samples_split=2, min_samples_leaf=4, max_features=0.2, max_depth=27, bootstrap=False, total=32.8min
[CV] n_estimators=90, min_samples_split=5, min_samples_leaf=2, max_features=0.2, max_depth=38, bootstrap=True 
[CV]  n_estimat